<a href="https://colab.research.google.com/github/VectorInstitute/Causal_Inference_Laboratory/blob/main/notebooks/Demo_DifferentLearners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo: Different Learners

## Different Learners:
- T learner (Grouped Conditional Outcome Modeling)
- S learner (Conditional Outcome Modeling)
- Inverse Propensity based Learner 

---

## Dataset
**IHDP Dataset: Simulated Outcome**
The dataset is from a randomized experiment studying the effect of home visits by specialists on future cognitive test scores of children. 

**Problem:** to measure the causal impact of home visits by specialists on children's cognitive score.

- The treatment is a binary variable: Home visits by specialists
- The dataset is split into train/test (672/75)
- The covariates come from a randomized experiment: there are 25 covariates that are the collected measurements of the children and their mothers
- Outcomes are simluated and the noiseless outcome is used as the ground truth

Each .npz file in the data folder contains the following keys: x, t, yf, ycf, mu0, mu1, which are respectively covariates, treatment, factual outcome, counterfactual outcome, noiseless potential control outcome, and noiseless potential treated outcome.

J. L. Hill, “Bayesian nonparametric modeling for causal inference,” Journal of Computational and Graphical Statistics, vol. 20, no. 1, pp. 217–240, 2011. [Online]. Available: https://doi.org/10.1198/jcgs.2010.08162


## Preparation
We will use some modules from our public repo on https://github.com/VectorInstitute/Causal_Inference_Laboratory.git

In [63]:
!git clone https://github.com/VectorInstitute/Causal_Inference_Laboratory.git
!mv Causal_Inference_Laboratory code
!mv code/data ./data
!mv code/utils ./utils
!mv code/models ./models
!mv code/estimation_results ./estimation_results

Cloning into 'Causal_Inference_Laboratory'...
remote: Enumerating objects: 328, done.
remote: Counting objects: 100% (328/328), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 328 (delta 135), reused 212 (delta 51), pack-reused 0
Receiving objects: 100% (328/328), 23.88 MiB | 19.73 MiB/s, done.
Resolving deltas: 100% (135/135), done.
mv: cannot move 'Causal_Inference_Laboratory' to 'code/Causal_Inference_Laboratory': Directory not empty
mv: cannot stat 'code/data': No such file or directory
mv: cannot stat 'code/utils': No such file or directory
mv: cannot stat 'code/models': No such file or directory
mv: cannot stat 'code/estimation_results': No such file or directory


In [64]:
!pip install xgboost==1.3.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [65]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from typing import Dict

from tensorflow import keras
from tensorflow.keras import layers, regularizers

import utils.estimators as models
import utils.preprocessing as helper
from utils.preprocessing import sys_config
import utils.metrics as metrics
from utils.evaluation import *

import warnings
warnings.filterwarnings('ignore')

In [66]:
# load the data
datasets_folder = sys_config["datasets_folder"]
results_folder = sys_config["results_folder"]

dataset_name = "IHDP-100" 
num_realizations = 100

x_train_all, t_train_all, yf_train_all = helper.load_IHDP_observational(
        datasets_folder, dataset_name, details=True
    )
x_test_all, t_test_all, yf_test_all = helper.load_IHDP_out_of_sample(
    datasets_folder, dataset_name, details=True
)

x_train, t_train, yf_train = x_train_all[:, :, :num_realizations], t_train_all[:, :num_realizations], yf_train_all[:, :num_realizations]
x_test = x_test_all[:, :, :num_realizations]
print(x_train[0, :, 0])

-------------------------------------------------------------------------------
The details of the train split of IHDP-100 dataset:
Number of realizations: 100
ate () int64
mu1 (672, 100) float64
mu0 (672, 100) float64
yadd () int64
yf (672, 100) float64
ycf (672, 100) float64
t (672, 100) float64
x (672, 25, 100) float64
ymul () int64
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
The details of the test split of IHDP-100 dataset:
Number of realizations: 100
ate () int64
mu1 (75, 100) float64
mu0 (75, 100) float64
yadd () int64
yf (75, 100) float64
ycf (75, 100) float64
t (75, 100) float64
x (75, 25, 100) float64
ymul () int64
-------------------------------------------------------------------------------
[ 0.31658816  0.59658219 -0.36089799 -0.87960599  0.37108604 -1.18901798
  1.          0.          1.          0.          1.          0.
  0.          2.          0.     

This is a semi-synthetic dataset and not fully observational. So. let's load the ground-truth that can be used for evaluations

In [67]:
# Load Ground Truth facutal and counterfactual outcomes for the training and test datasets
mu0_in, mu1_in, mu0_out, mu1_out = helper.load_IHDP_ground_truth(datasets_folder, dataset_name, details=False)

In [68]:
# let's use the first realization for now
i=0# No. of realization
x, t, yf = x_train_all[:, :, i], t_train_all[:, i], yf_train_all[:, i]
x_test, t_test, yf_test = x_test_all[:, :, i],t_test_all[:, i],yf_test_all[:, i]
mu0_in, mu1_in, mu0_out, mu1_out = mu0_in[:, i],mu1_in[:, i],mu0_out[:, i], mu1_out[:, i]

### T-Learner

In [69]:
def predict_by_GCOM(models, x, keras_model=False):
    """
    Grouped Conditional Outcome Modeling (GCOM), where for binary t, each t
    corresponds to one model. This is also called a S-learner.
    covariates
    :param models: two models for t=0 and t=1 separately
    :param x:
    :return:
    """
    if not keras_model:
        y0 = models[0].predict(x)
        y1 = models[1].predict(x)
    else:
        y0 = models[0].predict(x, verbose=0)
        y1 = models[1].predict(x, verbose=0)
    return y0, y1

model #1, Linear Regression


In [70]:
def train_and_estimate_OLS2(x, t, yf, x_test):
    """
    Training using Ordinary Least Squares (OLS). For binary t, each t
    corresponds to one model.
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_test: out-of-sample covariates
    :return:
    """
    mask = t == 0  # find the boolean mask of t is 0

    t0 = np.arange(len(t))[mask]
    t1 = np.arange(len(t))[~mask]

    x0 = x[t0]
    x1 = x[t1]

    # fit an estimator for each treatment separately
    OLS_t0 = LinearRegression()
    OLS_t0.fit(x0, yf[t0])
    OLS_t1 = LinearRegression()
    OLS_t1.fit(x1, yf[t1])

    y0_in, y1_in = predict_by_GCOM([OLS_t0, OLS_t1], x) # trainig set
    ate_in = np.mean(y1_in - y0_in) # ATE on training set
    y0_out, y1_out = predict_by_GCOM([OLS_t0, OLS_t1], x_test) # test set
    ate_out = np.mean(y1_out - y0_out) # ATE on testset
    return y0_in, y1_in, ate_in, y0_out, y1_out, ate_out


In [71]:
# Ordinery Least Square by Scikit Learn
(y0_in,y1_in,ate_in_pred,y0_out,y1_out,ate_out_pred) = train_and_estimate_OLS2(x, t, yf, x_test)

In [72]:
# evaluations
# calcualte PEHE
pehe_train = np.sqrt(mean_squared_error(mu1_in - mu0_in, y1_in - y0_in))
pehe_test = np.sqrt(mean_squared_error(mu1_out - mu0_out, y1_out - y0_out))

#calculate_mae_ATE (mean absolute error)
# Load ground truth ATE
ate_in_gt = np.mean(mu1_in - mu0_in)
ate_out_gt = np.mean(mu1_out - mu0_out)
mae_ATE_train = np.abs(ate_in_gt - ate_in_pred)
mae_ATE_test = np.abs(ate_out_gt - ate_out_pred)

print("PEHE_train:", pehe_train)
print("PEHE_test:", pehe_test)
print("mae_ATE_train:", mae_ATE_train)
print("mae_ATE_test:", mae_ATE_test)

PEHE_train: 0.6938747396144975
PEHE_test: 0.6658311473390662
mae_ATE_train: 0.051203444792220054
mae_ATE_test: 0.09020399498895992


model #2, Random Forest

In [73]:
def train_and_estimate_RF2(x, t, yf, x_test):
    """
    Training using Random Forest (RF). For binary t, each t
    corresponds to one model.
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_test: out-of-sample covariates
    :return:
    """
    mask = t == 0  # find the boolean mask of t is 0

    t0 = np.arange(len(t))[mask]
    t1 = np.arange(len(t))[~mask]

    x0 = x[t0]
    x1 = x[t1]

    # fit an estimator for each treatment separately
    RF_t0 = RandomForestRegressor()
    RF_t0.fit(x0, yf[t0])
    RF_t1 = RandomForestRegressor()
    RF_t1.fit(x1, yf[t1])

    y0_in, y1_in = predict_by_GCOM([RF_t0, RF_t1], x)
    ate_in = np.mean(y1_in - y0_in)
    y0_out, y1_out = predict_by_GCOM([RF_t0, RF_t1], x_test)
    ate_out = np.mean(y1_out - y0_out)
    return y0_in, y1_in, ate_in, y0_out, y1_out, ate_out


In [74]:
(y0_in,y1_in,ate_in_pred,y0_out,y1_out,ate_out_pred) = train_and_estimate_RF2(x, t, yf, x_test)

# evaluations
# calcualte PEHE
pehe_train = np.sqrt(mean_squared_error(mu1_in - mu0_in, y1_in - y0_in))
pehe_test = np.sqrt(mean_squared_error(mu1_out - mu0_out, y1_out - y0_out))

#calculate_mae_ATE
mae_ATE_train = np.abs(ate_in_gt - ate_in_pred)
mae_ATE_test = np.abs(ate_out_gt - ate_out_pred)

print("PEHE_train:", pehe_train)
print("PEHE_test:", pehe_test)
print("mae_ATE_train:", mae_ATE_train)
print("mae_ATE_test:", mae_ATE_test)

PEHE_train: 0.8719083513833528
PEHE_test: 0.6309037582830833
mae_ATE_train: 0.11357146452698652
mae_ATE_test: 0.048904717346568205




---

### S learner

In [75]:
def predict_by_COM(model, x, keras_model=False):
    """
    Conditional Outcome Modeling (COM), where t is viewed as one feature of the
    covariates. This is also called a T-learner.
    :param model:
    :param x:
    :return:
    """
    if not keras_model:
        y0 = model.predict(
            np.concatenate((x, np.zeros(shape=(x.shape[0], 1))), axis=1)
        )
        y1 = model.predict(
            np.concatenate((x, np.ones(shape=(x.shape[0], 1))), axis=1)
        )
    else:
        y0 = model.predict(
            np.concatenate((x, np.zeros(shape=(x.shape[0], 1))), axis=1),
            verbose=0,
        )
        y1 = model.predict(
            np.concatenate((x, np.ones(shape=(x.shape[0], 1))), axis=1),
            verbose=0,
        )
    return y0, y1

model #1, Linear Regression

In [76]:
def train_and_estimate_OLS1(x, t, yf, x_test):
    """
    Training using Ordinary Least Squares (OLS), where t is viewed as one
    feature of the covariates
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_test: out-of-sample covariates
    :return:
    """

    new_x = np.concatenate((x, t[:, np.newaxis]), axis=1)
    OLS = LinearRegression()
    OLS.fit(new_x, yf)

    y0_in, y1_in = predict_by_COM(OLS, x)
    ate_in = np.mean(y1_in - y0_in)
    y0_out, y1_out = predict_by_COM(OLS, x_test)
    ate_out = np.mean(y1_out - y0_out)
    return y0_in, y1_in, ate_in, y0_out, y1_out, ate_out


In [77]:
(y0_in,y1_in,ate_in_pred,y0_out,y1_out,ate_out_pred) = train_and_estimate_OLS1(x, t, yf, x_test)

# evaluations
# calcualte PEHE
pehe_train = np.sqrt(mean_squared_error(mu1_in - mu0_in, y1_in - y0_in))
pehe_test = np.sqrt(mean_squared_error(mu1_out - mu0_out, y1_out - y0_out))

#calculate_mae_ATE
mae_ATE_train = np.abs(ate_in_gt - ate_in_pred)
mae_ATE_test = np.abs(ate_out_gt - ate_out_pred)

print("PEHE_train:", pehe_train)
print("PEHE_test:", pehe_test)
print("mae_ATE_train:", mae_ATE_train)
print("mae_ATE_test:", mae_ATE_test)

PEHE_train: 0.8685046985895427
PEHE_test: 0.8022900974955098
mae_ATE_train: 0.06646990254738983
mae_ATE_test: 0.11124416777178237


model #2, Random Forest

In [78]:
def train_and_estimate_RF1(x, t, yf, x_test):
    """
    Training using Random Forest (RF), where t is viewed as one
    feature of the covariates
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_test: out-of-sample covariates
    :return:
    """

    new_x = np.concatenate((x, t[:, np.newaxis]), axis=1)
    RF = RandomForestRegressor()
    RF.fit(new_x, yf)

    y0_in, y1_in = predict_by_COM(RF, x)
    ate_in = np.mean(y1_in - y0_in)
    y0_out, y1_out = predict_by_COM(RF, x_test)
    ate_out = np.mean(y1_out - y0_out)
    return y0_in, y1_in, ate_in, y0_out, y1_out, ate_out


In [79]:
(y0_in,y1_in,ate_in_pred,y0_out,y1_out,ate_out_pred) = train_and_estimate_RF1(x, t, yf, x_test)

# evaluations
# calcualte PEHE
pehe_train = np.sqrt(mean_squared_error(mu1_in - mu0_in, y1_in - y0_in))
pehe_test = np.sqrt(mean_squared_error(mu1_out - mu0_out, y1_out - y0_out))

#calculate_mae_ATE
mae_ATE_train = np.abs(ate_in_gt - ate_in_pred)
mae_ATE_test = np.abs(ate_out_gt - ate_out_pred)

print("PEHE_train:", pehe_train)
print("PEHE_test:", pehe_test)
print("mae_ATE_train:", mae_ATE_train)
print("mae_ATE_test:", mae_ATE_test)

PEHE_train: 0.8688918673489963
PEHE_test: 0.6273430493843163
mae_ATE_train: 0.09065304952568098
mae_ATE_test: 0.025418499152054963


model #3: Neural Network

In [80]:
def create_simple_NN(input_dim):
    # model = keras.Sequential(
    #     [
    #         layers.InputLayer(input_shape=(input_dim,), name="input_layer"),
    #         layers.Dense(input_dim, activation="relu", name="first_layer"),
    #         layers.Dense(1, name="output_layer"),
    #     ]
    # )

    # use the backbone NN from TAR-Net
    model = keras.Sequential(
        [
            layers.InputLayer(input_shape=(input_dim,), name="input_layer"),
            layers.Dense(
                units=200, activation="elu", kernel_initializer="RandomNormal"
            ),
            layers.Dense(
                units=200, activation="elu", kernel_initializer="RandomNormal"
            ),
            layers.Dense(
                units=200, activation="elu", kernel_initializer="RandomNormal"
            ),
            layers.Dense(1, name="output_layer"),
        ]
    )
    return model


def train_and_estimate_NN1(x, t, yf, x_test):
    """
    Training using simple fully-connected neural networks, where t is viewed as
    one feature of the covariates
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_test: out-of-sample covariates
    :return:
    """

    new_x = np.concatenate((x, t[:, np.newaxis]), axis=1)
    model = create_simple_NN(new_x.shape[-1])
    model.compile(loss="mean_squared_error", optimizer="adam", metrics="mse")
    model.fit(new_x, yf, epochs=50, verbose=0)
    y0_in, y1_in = predict_by_COM(model, x, keras_model=True)
    ate_in = np.mean(y1_in - y0_in)
    y0_out, y1_out = predict_by_COM(model, x_test, keras_model=True)
    ate_out = np.mean(y1_out - y0_out)
    return y0_in, y1_in, ate_in, y0_out, y1_out, ate_out

In [81]:
(y0_in,y1_in,ate_in_pred,y0_out,y1_out,ate_out_pred) = train_and_estimate_NN1(x, t, yf, x_test)

# evaluations
# calcualte PEHE
pehe_train = np.sqrt(mean_squared_error(mu1_in - mu0_in, y1_in - y0_in))
pehe_test = np.sqrt(mean_squared_error(mu1_out - mu0_out, y1_out - y0_out))

#calculate_mae_ATE
mae_ATE_train = np.abs(ate_in_gt - ate_in_pred)
mae_ATE_test = np.abs(ate_out_gt - ate_out_pred)

print("PEHE_train:", pehe_train)
print("PEHE_test:", pehe_test)
print("mae_ATE_train:", mae_ATE_train)
print("mae_ATE_test:", mae_ATE_test)

PEHE_train: 0.3410017658001972
PEHE_test: 0.31658499141309787
mae_ATE_train: 0.062119803011495556
mae_ATE_test: 0.0531752629227773




---
### Inverse Propensity based Learners

In [82]:
def train_and_estimate_IPW(x, t, yf, x_test,t_test,yf_test):
    """
    Training using Inverse Probability Weighting (IPW). For binary t, each t
    corresponds to one model.
    :param x: covariates
    :param t: treatment
    :param yf: factual outcomes
    :param x_test: out-of-sample covariates
    :return:
    """
    LR = LogisticRegression()
    LR.fit(x, t)
    propensity = LR.predict_proba(x)[:, 1]  # T = 1

    mask = t == 0  # find the boolean mask of t is 0
    t0 = np.arange(len(t))[mask]
    t1 = np.arange(len(t))[~mask]

    yf0 = yf[t0]
    denominator0 = (1 - propensity)[t0]
    yf1 = yf[t1]
    denominator1 = propensity[t1]
    ate_in = np.mean(yf1 / denominator1) - np.mean(yf0 / denominator0)

    propensity_out = LR.predict_proba(x_test)[:, 1]
    mask = t_test == 0  # find the boolean mask of t is 0
    t0_test = np.arange(len(t_test))[mask]
    t1_test = np.arange(len(t_test))[~mask]

    yf0_test = yf_test[t0_test]
    denominator0_out = (1 - propensity_out)[t0_test]
    yf1_test = yf_test[t1_test]
    denominator1_out = propensity_out[t1_test]
    ate_out = np.mean(yf1_test / denominator1_out) - np.mean(yf0_test / denominator0_out)
    
    return ate_in, ate_out

In [83]:
ate_in_pred,ate_out_pred = train_and_estimate_IPW(x, t, yf, x_test,t_test,yf_test)
print([ate_in_pred,ate_out_pred])

[28.645137430276893, 38.46448326173249]




---
##  All realizations (using T Learner)

In [84]:
#Let's load the data again
dataset_name = "IHDP-100"
# Load covariates, treatment, and factual outcomes for the training and test datasets
x_train_all, t_train_all, yf_train_all = helper.load_IHDP_observational(datasets_folder, dataset_name, details=False)
x_test_all, t_test_all, yf_test_all = helper.load_IHDP_out_of_sample(datasets_folder, dataset_name, details=False)
# Load Ground Truth facutal and counterfactual outcomes for the training and test datasets
mu0_in, mu1_in, mu0_out, mu1_out = helper.load_IHDP_ground_truth(datasets_folder, dataset_name, details=False)
# Load ground truth ATE
ate_in_gt = np.mean(mu1_in - mu0_in)
ate_out_gt = np.mean(mu1_out - mu0_out)

In [85]:
def estimate(estimator_name):
    num_realizations = x_train_all.shape[-1]
    print("Numer of realizations:", num_realizations)
    y0_in_all, y1_in_all, y0_out_all, y1_out_all = [], [], [], []
    ate_in_all, ate_out_all = [], []
    for i in range(num_realizations):
        text = f" Estimation of realization {i} via {estimator_name}"
        print(f"{text:-^79}")
        x, t, yf = x_train_all[:, :, i], t_train_all[:, i], yf_train_all[:, i]
        x_test = x_test_all[:, :, i]
        # train the estimator and predict for this realization
        (
            y0_in,
            y1_in,
            ate_in,
            y0_out,
            y1_out,
            ate_out,
        ) = models.train_and_evaluate(x, t, yf, x_test, estimator_name, dataset_name)
        y0_in_all.append(y0_in)
        y1_in_all.append(y1_in)
        ate_in_all.append(ate_in)
        y0_out_all.append(y0_out)
        y1_out_all.append(y1_out)
        ate_out_all.append(ate_out)
    # follow the dimension order of the dataset,
    # i.e., realizations are captured by the last index
    y0_in_all = np.squeeze(np.array(y0_in_all).transpose()).reshape((-1, num_realizations))
    y1_in_all = np.squeeze(np.array(y1_in_all).transpose()).reshape((-1, num_realizations))
    y0_out_all = np.squeeze(np.array(y0_out_all).transpose()).reshape((-1, num_realizations))
    y1_out_all = np.squeeze(np.array(y1_out_all).transpose()).reshape((-1, num_realizations))
    ate_in_all = np.array(ate_in_all).reshape((num_realizations,))
    ate_out_all = np.array(ate_out_all).reshape((num_realizations,))

    # save estimation results
    estimation_result_folder = os.path.join(
        results_folder, dataset_name, estimator_name
    )
    return y0_in_all,y1_in_all,y0_out_all,y1_out_all,ate_in_all,ate_out_all

In [86]:
estimator = 'OLS1'
y0_in_all,y1_in_all,y0_out_all,y1_out_all,ate_in_all,ate_out_all=estimate(estimator)

Numer of realizations: 100
--------------------- Estimation of realization 0 via OLS1---------------------
--------------------- Estimation of realization 1 via OLS1---------------------
--------------------- Estimation of realization 2 via OLS1---------------------
--------------------- Estimation of realization 3 via OLS1---------------------
--------------------- Estimation of realization 4 via OLS1---------------------
--------------------- Estimation of realization 5 via OLS1---------------------
--------------------- Estimation of realization 6 via OLS1---------------------
--------------------- Estimation of realization 7 via OLS1---------------------
--------------------- Estimation of realization 8 via OLS1---------------------
--------------------- Estimation of realization 9 via OLS1---------------------
-------------------- Estimation of realization 10 via OLS1---------------------
-------------------- Estimation of realization 11 via OLS1---------------------
-------------

In [87]:
metrics_set = ["PEHE", "MAE"]
results_in={}
for metric in metrics_set:
  metric_in = metrics.calculate_metrics(y0_in_all, y1_in_all, ate_in_all, mu0_in, mu1_in, ate_in_gt, metric=metric)
  results_in[metric] = {"mean": np.mean(metric_in, where=(metric_in != 0))}

In [88]:
results_in

{'PEHE': {'mean': 5.600176017623678}, 'MAE': {'mean': 0.7463799989764288}}



---

**Different Estimators and All realizations**

In [89]:
# T Learners: OLS2, RF2, NN2
# S Learners: OLS1, RF1, NN1

In [ ]:
estimator_set = ["OLS1", "OLS2", "RF1", "RF2","NN1","NN2"]
metrics_set = ["PEHE", "MAE"]
results_in = pd.DataFrame(index=estimator_set,columns=metrics_set)
for estimator_name in estimator_set:
  print (estimator_name)
  y0_in_all,y1_in_all,y0_out_all,y1_out_all,ate_in_all,ate_out_all=estimate(estimator_name)

  # evaluation
  #results_in[estimator] = {}
  for metric in metrics_set:
    metric_in = metrics.calculate_metrics(y0_in_all, y1_in_all, ate_in_all, mu0_in, mu1_in, ate_in_gt, metric=metric)
    results_in.loc[estimator_name,metric]= np.mean(metric_in, where=(metric_in != 0))


OLS1
Numer of realizations: 100
--------------------- Estimation of realization 0 via OLS1---------------------
--------------------- Estimation of realization 1 via OLS1---------------------
--------------------- Estimation of realization 2 via OLS1---------------------
--------------------- Estimation of realization 3 via OLS1---------------------
--------------------- Estimation of realization 4 via OLS1---------------------
--------------------- Estimation of realization 5 via OLS1---------------------
--------------------- Estimation of realization 6 via OLS1---------------------
--------------------- Estimation of realization 7 via OLS1---------------------
--------------------- Estimation of realization 8 via OLS1---------------------
--------------------- Estimation of realization 9 via OLS1---------------------
-------------------- Estimation of realization 10 via OLS1---------------------
-------------------- Estimation of realization 11 via OLS1---------------------
--------

In [91]:
results_in


,PEHE,MAE
OLS1,5.600176,0.74638
OLS2,2.235139,0.135197
RF1,3.016836,0.475274
RF2,1.720281,0.11651
